In [79]:
import torch 
import numpy as np 
import argparse
from torch_geometric.loader import GraphSAINTRandomWalkSampler, GraphSAINTNodeSampler, GraphSAINTEdgeSampler
from torch_geometric.nn import GraphConv
from torch_geometric.utils import degree
import torch.nn.functional as F
import os
from torch_sparse import spmm
import dataset
import models

In [3]:
from torch_geometric.data import GraphSAINTSampler

In [8]:
# batch size: The approximate number of samples per batch.
# walk_length=2
# num_steps (int, optional): The number of iterations per epoch. (how many times the weights are updated), and they are updated once per subgraph.
# sample_coverage (int): How many samples per node should be used to compute normalization statistics. (default: :obj:`0`)

In [80]:
path = f"./dataset/Flickr"
dataset = dataset.load_dataset("Flickr", path)

data = dataset[0]

In [64]:
dataset[0]

Data(x=[89250, 500], edge_index=[2, 899756], y=[89250], train_mask=[89250], val_mask=[89250], test_mask=[89250])

In [81]:
node_loader = GraphSAINTNodeSampler(data, batch_size=6000, num_steps=5, sample_coverage=100,
                                     save_dir=dataset.processed_dir,
                                     num_workers=0)

In [90]:
loader = torch.utils.data.DataLoader(node_loader, batch_size=200)

In [91]:
for i, data in enumerate(loader):
    print(data)
    break

RuntimeError: stack expects each tensor to be equal size, but got [5188] at entry 0 and [5200] at entry 1

In [92]:
for data in loader:
    for node_idx, adj in data:
        print(node_idx)
        print(adj)
        edge_idx = adj.storage.value()
        node_count[node_idx] += 1
        edge_count[edge_idx] += 1
        total_sampled_nodes += node_idx.size(0)
        print(node_idx)
        print(adj)
        break
    break

RuntimeError: stack expects each tensor to be equal size, but got [5261] at entry 0 and [5217] at entry 1

In [18]:
loader = GraphSAINTRandomWalkSampler(data, batch_size=6000, walk_length=2,
                                     num_steps=5, sample_coverage=100,
                                     save_dir=dataset.processed_dir,
                                     num_workers=0)

In [30]:
loader.adj

SparseTensor(row=tensor([    0,     0,     0,  ..., 89249, 89249, 89249]),
             col=tensor([    1,  1694,  2569,  ..., 28631, 52736, 78119]),
             val=tensor([     0,      1,      2,  ..., 899753, 899754, 899755]),
             size=(89250, 89250), nnz=899756, density=0.01%)

In [29]:
for data in loader:
    print(data)
#     break

Data(num_nodes=14459, edge_index=[2, 85844], x=[14459, 500], y=[14459], train_mask=[14459], val_mask=[14459], test_mask=[14459], node_norm=[14459], edge_norm=[85844])
Data(num_nodes=14400, edge_index=[2, 83478], x=[14400, 500], y=[14400], train_mask=[14400], val_mask=[14400], test_mask=[14400], node_norm=[14400], edge_norm=[83478])
Data(num_nodes=14536, edge_index=[2, 85902], x=[14536, 500], y=[14536], train_mask=[14536], val_mask=[14536], test_mask=[14536], node_norm=[14536], edge_norm=[85902])
Data(num_nodes=14523, edge_index=[2, 86850], x=[14523, 500], y=[14523], train_mask=[14523], val_mask=[14523], test_mask=[14523], node_norm=[14523], edge_norm=[86850])
Data(num_nodes=14438, edge_index=[2, 86144], x=[14438, 500], y=[14438], train_mask=[14438], val_mask=[14438], test_mask=[14438], node_norm=[14438], edge_norm=[86144])


(tensor([    2,     5,     6,  ..., 89225, 89229, 89238]),
 SparseTensor(row=tensor([    0,     0,     0,  ..., 14558, 14558, 14558]),
              col=tensor([ 71, 161, 168,  ..., 203, 337, 562]),
              val=tensor([    40,     41,     42,  ..., 899696, 899697, 899698]),
              size=(14559, 14559), nnz=86530, density=0.04%))

In [ ]:
class MetropolisHastingsSampler(GraphSAINTSampler):
    r"""The GraphSAINT random walk sampler class (see
    :class:`torch_geometric.data.GraphSAINTSampler`).

    Args:
        budget (int): The number of actions tracking an edge is denoted as total budget, 
        which is denoted as 𝐵 here. Usually, 𝐵 ≥ |𝑉′| as RW-based algorithms are likely 
        to backtrack when exploring the original graph.
    """
    def __init__(self, data, batch_size: int, budget: int,
                 num_steps: int = 1, sample_coverage: int = 0,
                 save_dir: Optional[str] = None, log: bool = True, **kwargs):
        self.walk_length = walk_length
        super(GraphSAINTRandomWalkSampler,
              self).__init__(data, batch_size, num_steps, sample_coverage,
                             save_dir, log, **kwargs)

    @property
    def __filename__(self):
        return (f'{self.__class__.__name__.lower()}_{self.walk_length}_'
                f'{self.sample_coverage}.pt')

    def __sample_nodes__(self, batch_size):
        start = torch.randint(0, self.N, (batch_size, ), dtype=torch.long)
        node_idx = self.adj.random_walk(start.flatten(), self.walk_length)
        return node_idx.view(-1)


In [31]:
start = torch.randint(0, loader.N, (5, ), dtype=torch.long)

In [34]:
node_idx = loader.adj.random_walk(start.flatten(), loader.walk_length)

In [40]:
node_idx.view(-1)(6000).unique()

TypeError: 'Tensor' object is not callable

In [68]:
edge_sample = torch.randint(0, node_loader.E, (6000, 6000),
                                    dtype=torch.long)

In [69]:
edge_sample

tensor([[729717, 440086, 868947,  ..., 709346, 715842, 216320],
        [340817, 858222, 486669,  ..., 803033, 273956, 704255],
        [276650, 484040, 347149,  ..., 443384, 677804,  76012],
        ...,
        [871741, 116139, 779614,  ..., 509843, 423690, 601633],
        [636155, 753056, 502312,  ..., 471880, 879143, 646626],
        [683955, 519491, 847756,  ..., 373889,   4157, 841058]])

In [75]:
node_loader.sample_nodes(6000)

AttributeError: 'GraphSAINTNodeSampler' object has no attribute 'sample_nodes'